Задание из видео Карпова [Разбор тестового задания на позицию junior аналитика данных](https://www.youtube.com/watch?v=3VWDh-gdiYg)

Описание данных:  
orders - таблица с информацией о заказах;  
order_id - уникальный идентификатор заказа;  
customer_id - уникальный идентификатор клиента;  
order_date - дата, когда заказ был создан;  
order_amount - cумма заказа;  
platform - платформа, через которую был сделан заказ (например, 'mobile', 'desktop');  
category - категория товара, который был заказан (например, 'smartphones', 'laptops').

1. Построй когортый анализ: выдели клиентов, совершивших первую покупку в течение определенного месяца (когортную дату) и вычисли размер каждой когорты.  
2. Рассчитай средний джоход с клиента для каждой когорты за 10 дней с момента их первой покупки.  
3. Отобрази результаты по месяцам первых покупок, категориям товаров и платформам.  

Вывод должен содержать следующие поля:
- дата когорты;
- платформа;
- категория товара;
- размер когорты;
- средний доход с клиента.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [116]:
customer_ids = np.array([])
for i in range(1,14):
    customer_ids = np.append(customer_ids, np.random.choice(range(1, i*10 + 1), (1, i * 101)))

In [117]:
customer_ids = customer_ids.astype(int)

In [118]:
# число заказов
N = customer_ids.size

In [119]:
order_dates = pd.date_range('2023-01-01', '2024-01-31', periods=N)

In [120]:
order_amounts = 111 + np.random.exponential(1, N) * 10000

In [121]:
platforms = np.random.choice(['mobile', 'desktop', 'shop'], N, p=[0.5, 0.3, 0.2])

In [122]:
categories = np.random.choice(['smartphones', 'laptops', 'watches'], N, p=[0.5, 0.3, 0.2])

In [123]:
df = pd.DataFrame({
    'order_id': range(1, N + 1), 
    'customer_id': customer_ids, 
    'order_date': order_dates, 
    'order_amount': order_amounts, 
    'platform': platforms, 
    'category': categories
})

In [124]:
df.dtypes

order_id                 int64
customer_id              int32
order_date      datetime64[ns]
order_amount           float64
platform                object
category                object
dtype: object

In [125]:
df.order_id = df.order_id.astype('int32')

In [126]:
df.platform = df.platform.astype('category')

In [127]:
df.category = df.category.astype('category')

In [128]:
df.dtypes

order_id                 int32
customer_id              int32
order_date      datetime64[ns]
order_amount           float64
platform              category
category              category
dtype: object

In [98]:
# df.to_pickle('data.pkl')

In [129]:
df['order_day'] = df.order_date.dt.to_period('D')

In [130]:
df['order_month'] = df.order_date.dt.to_period('M')

In [139]:
df['day_from_start'] = df['order_day'].astype(int) - df.groupby('customer_id')['order_day'].transform('min').astype(int)
df['day_from_start'] = df['day_from_start'].astype('int32')

In [142]:
df.sample(20)

,order_id,customer_id,order_date,order_amount,platform,category,order_day,order_month,day_from_start
7891,7892,45,2023-12-06 04:00:31.338411316,3874.697454,desktop,laptops,2023-12-06,2023-12,295
5763,5764,49,2023-09-05 16:51:26.833514692,1504.531933,desktop,laptops,2023-09-05,2023-09,204
4710,4711,26,2023-07-22 10:37:44.200217628,16451.664786,desktop,watches,2023-07-22,2023-07,189
4679,4680,19,2023-07-21 02:39:02.546245920,7672.419782,shop,laptops,2023-07-21,2023-07,196
5306,5307,78,2023-08-17 01:26:10.837867248,3187.961460,mobile,laptops,2023-08-17,2023-08,102
3459,3460,47,2023-05-29 16:09:08.422198042,6364.046302,mobile,smartphones,2023-05-29,2023-05,104
2187,2188,61,2023-04-05 00:00:47.007616975,9010.130680,mobile,smartphones,2023-04-05,2023-04,0
345,346,21,2023-01-15 19:53:12.600652883,1557.682790,desktop,smartphones,2023-01-15,2023-01,0
2292,2293,26,2023-04-09 12:19:35.190424375,8964.691977,mobile,smartphones,2023-04-09,2023-04,85
8912,8913,87,2024-01-19 01:13:38.715995648,11467.879253,shop,laptops,2024-01-19,2024-01,225
